# Importaciones

In [3]:
import json 
from datetime import datetime
import re

import pandas as pd 
import numpy as np

from googleapiclient.errors import HttpError
from Google import Create_Service

# Obtención de datos

## Llamamiento a la API de YouTube

Para obtener los datos vamos a usar las descripciones en cada video de las mañaneras. Pues nos provee de un resumen del video del día. De esta manera ya no tenemos que lidiar con conversaciones del presidente cuando usamos las transcripciones.

Función que utiliza expresiones regulares para obtener los segundos totales de una duración dada.

In [3]:
def convert_duration(duration):
    try:
        h = int(re.search('\d+H', duration)[0][:-1]) * 60**2  if re.search('\d+H', duration) else 0 # hour
        m = int(re.search('\d+M', duration)[0][:-1]) * 60  if re.search('\d+M', duration) else 0 # minute
        s = int(re.search('\d+S', duration)[0][:-1])  if re.search('\d+S', duration) else 0 # second
        return h + m + s
    except Exception as e:
        print(e)
        return 0 

In [4]:
def retrieve_playlists(service, channelId):
    playlists =[]
    try:
        response = service.playlists().list(
            part = 'contentDetails, snippet, status',
            channelId = channelId,
            maxResults = 50
        ).execute()

        playlists.extend(response.get('items'))
        nextPageToken = response.get('nextPageToken')

        while nextPageToken:
            response = service.playlists().list(
                part = 'contentDetails, snippet, status',
                channelId = channelId,
                maxResults = 50,
                pageToken = nextPageToken
            ).execute()

            playlists.extend(response.get('items'))
            nextPageToken = response.get('nextPageToken')

        return playlists

    except HttpError as e:
        errMsg = json.loads(e.content)
        print("HTTP Error: ")
        print(errMsg['error']['message'])
        return []

Función que hace la petición a la API de youtube

In [5]:
def retrieve_playlists_items(service, playlistId):
    items =[]
    try:
        response = service.playlistItems().list(
            part = 'contentDetails, snippet, status',
            playlistId = playlistId,
            maxResults = 5
        ).execute()

        items.extend(response.get('items'))
        nextPageToken = response.get('nextPageToken')

        while nextPageToken:
            response = service.playlistItems().list(
                part = 'contentDetails, snippet, status',
                playlistId = playlistId,
                maxResults = 5,
                pageToken = nextPageToken
            ).execute()

            items.extend(response.get('items'))
            nextPageToken = response.get('nextPageToken')

        return items

    except HttpError as e:
        errMsg = json.loads(e.content)
        print("HTTP Error: ")
        print(errMsg['error']['message'])
        return []

Creando constantes para llamar a la API

In [6]:
CLIENT_SECRETS_FILE = 'client_secret.json'  # Archivo .json que se obtiene al crear las credenciales OAuth en https://console.cloud.google.com/
API_NAME = 'youtube'
API_VERSION = 'v3'
SCOPES = ['https://www.googleapis.com/auth/youtube.readonly']

service = Create_Service(CLIENT_SECRETS_FILE, API_NAME, API_VERSION, SCOPES)

client_secret.json-youtube-v3-(['https://www.googleapis.com/auth/youtube.readonly'],)
['https://www.googleapis.com/auth/youtube.readonly']
youtube service created successfully


El objeto <code>service</code> nos da accesos a todos los recursos de la API

In [7]:
dir(service)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_basic_methods',
 '_add_nested_resources',
 '_add_next_methods',
 '_baseUrl',
 '_developerKey',
 '_dynamic_attrs',
 '_http',
 '_model',
 '_requestBuilder',
 '_resourceDesc',
 '_rootDesc',
 '_schema',
 '_set_dynamic_attr',
 '_set_service_methods',
 'abuseReports',
 'activities',
 'captions',
 'channelBanners',
 'channelSections',
 'channels',
 'close',
 'commentThreads',
 'comments',
 'i18nLanguages',
 'i18nRegions',
 'liveBroadcasts',
 'liveChatBans',
 'liveChatMessages',
 'liveChatModerators',
 'liveStreams',
 'members',
 'membershipsLevels',
 'new_batch_h

Hacemos una llamada a a función de <code>retrieve_playlists_items</code> para obtener todos los videos una playlist especifica 

In [47]:
playlistId = 'PLRnlRGar-_296KTsVL0R6MEbpwJzD8ppA' # id de la playlist de las mañaneras
playlists_items = retrieve_playlists_items(service, playlistId)

In [9]:
playlists_items[0] # El último video agregado a la playlist

{'kind': 'youtube#playlistItem',
 'etag': 'c_spoCV2wb1dgyoY8MZpxa_qqb8',
 'id': 'UExSbmxSR2FyLV8yOTZLVHNWTDBSNk1FYnB3SnpEOHBwQS43OEJDMUVGM0VCNUQwREIw',
 'snippet': {'publishedAt': '2023-04-26T06:07:35Z',
  'channelId': 'UCxEgOKuI-n-WOJaNcisHvSg',
  'title': 'Conferencia de prensa matutina, desde Palacio Nacional. Miércoles 26 de abril 2023 | Presidente AMLO',
  'description': 'Conferencia de prensa matutina, desde Palacio Nacional. Miércoles 26 de abril 2023 | Presidente AMLO.\n\nSigue las actividades del presidente de México:\n\nSitio web:\nhttp://presidente.gob.mx\n\nYouTube:\nhttps://www.youtube.com/lopezobrador\n\nFacebook:\nhttps://facebook.com/lopezobrador.org.mx\n\nTwitter:\nhttps://twitter.com/lopezobrador_\n\nInstagram:\nhttps://www.instagram.com/lopezobrador/\n\nTelegram: \nhttps://t.me/PresidenteAMLO\n\nSpotify:\nhttps://spoti.fi/3QocikN',
  'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/25tJB6UoTzo/default.jpg',
    'width': 120,
    'height': 90},
   'medium': {

## Guardando los datos en un DataFrame

Podemos obtener cada elemento de la variable utilizando las posiciones.

In [10]:
playlists_items[0]['snippet']

{'publishedAt': '2023-04-26T06:07:35Z',
 'channelId': 'UCxEgOKuI-n-WOJaNcisHvSg',
 'title': 'Conferencia de prensa matutina, desde Palacio Nacional. Miércoles 26 de abril 2023 | Presidente AMLO',
 'description': 'Conferencia de prensa matutina, desde Palacio Nacional. Miércoles 26 de abril 2023 | Presidente AMLO.\n\nSigue las actividades del presidente de México:\n\nSitio web:\nhttp://presidente.gob.mx\n\nYouTube:\nhttps://www.youtube.com/lopezobrador\n\nFacebook:\nhttps://facebook.com/lopezobrador.org.mx\n\nTwitter:\nhttps://twitter.com/lopezobrador_\n\nInstagram:\nhttps://www.instagram.com/lopezobrador/\n\nTelegram: \nhttps://t.me/PresidenteAMLO\n\nSpotify:\nhttps://spoti.fi/3QocikN',
 'thumbnails': {'default': {'url': 'https://i.ytimg.com/vi/25tJB6UoTzo/default.jpg',
   'width': 120,
   'height': 90},
  'medium': {'url': 'https://i.ytimg.com/vi/25tJB6UoTzo/mqdefault.jpg',
   'width': 320,
   'height': 180},
  'high': {'url': 'https://i.ytimg.com/vi/25tJB6UoTzo/hqdefault.jpg',
   'wi

Hay que crear un dataframe con la información necesaria para hacer el entrenamiento. Estos datos son: EL título, la descrición y la fecha de publicación

In [48]:

data = []

for item in playlists_items:
    snippet = item['snippet']
    published_at = snippet['publishedAt']
    title = snippet['title']
    description = snippet['description']
    
    data.append([published_at, title, description])
    
df = pd.DataFrame(data, columns=['published_at', 'title', 'description'])
df = df.head(1087)

print(df)


              published_at                                              title   
0     2023-04-26T06:07:35Z  Conferencia de prensa matutina, desde Palacio ...  \
1     2023-04-25T04:45:24Z  Conferencia de prensa matutina, desde Palacio ...   
2     2023-04-24T12:44:41Z  Conferencia de prensa matutina desde Palacio N...   
3     2023-04-21T05:16:34Z  Seguridad y bienestar mejoran en Veracruz. Con...   
4     2023-04-20T11:50:16Z  Recursos de venta de avión serán para construi...   
...                    ...                                                ...   
1082  2019-03-08T21:59:25Z  Conferencia matutina: Avances del Gabinete de ...   
1083  2019-03-08T21:59:25Z  Hoy anunciamos la terna de candidatos a minist...   
1084  2019-03-08T21:59:25Z  Tercera conferencia de prensa matutina desde P...   
1085  2019-03-08T21:59:25Z     Diálogo con periodistas desde Palacio Nacional   
1086  2019-03-08T21:59:25Z  Primera conferencia de prensa matutina desde P...   

                           

## Una primera limpieza

Los datos que llegan de la API están contaminados con datos como minutos de YouTube en formato <code>00:00</code>, links de youtube y palabras específicas como nombres redes sociales y frases que no aportan nada.
Para mejor ejemplo, tomemos un elemento.

In [39]:
df['description'][500]

'Conferencia de prensa matutina, desde Campeche. Jueves 25 de marzo 2021 | Presidente AMLO.\n\n0:00 Inicio de transmisión\n11:27 Comienza la conferencia de prensa del presidente Andrés Manuel López Obrador\n26:21 Informe de seguridad del estado de Campeche\n48:41 Sesión de preguntas y respuestas\n\nRealizamos la conferencia de prensa matutina y la reunión del Gabinete de Seguridad en Campeche, la entidad federativa con menor incidencia delictiva del país. Esto se debe al trabajo de gobierno, al apoyo de la población, a la coordinación entre autoridades y a la atención a las causas de la violencia e inseguridad.\n\nDe acuerdo con el análisis del último año, en Campeche disminuyó la incidencia delictiva total en 14.41 por ciento, el feminicidio en 57.14 por ciento, la extorsión en 48 por ciento, el robo de vehículo en 38.89 por ciento, la violencia familiar en 31.58 por ciento, el robo a transeúnte en 2.44 por ciento y la trata de personas en un 100 por ciento. \n\nEn los delitos del fue

Declararemos una serie de funciones que utilizarán regex para limpiar esta basura.

<code>replace_newlines(string)</code> es una función que elimina los patrones de salto de línea (<code>\n</code>) y coloca un espacio en blanco

In [31]:
def replace_newlines(string):
    pattern = r'\n'
    replace_with = ' '
    return re.sub(pattern, replace_with, string)

<code>replace_minutes(string)</code> es una función que elimina los patrones de formato de minuto (<code>00:00</code>)

In [32]:
def remove_minutes(string):
    pattern = r'[0-9]?[0-9]:[0-9][0-9]'
    replace_with = ''
    return re.sub(pattern, replace_with, string)

<code>replace_minutes(string)</code> es una función que elimina los patrones de formato de minuto (<code>00:00</code>)

In [15]:
def remove_links(string):
    pattern = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    replace_with = ''
    return re.sub(pattern, replace_with, string)

<code>remove_extra_words(string)</code> es una función que elimina los patrones de palabras especícas, pues estas sólo pueden afectar el entrenamiento

In [16]:
def remove_extra_words(string):
    pattern = r' Sigue las actividades del presidente de México:  Sitio web:   YouTube:   Facebook:   Twitter:   Instagram:   Telegram:    Spotify: '
    replace_with = ''
    return re.sub(pattern, replace_with, string)

Aplicamos las funciones y vemos los resultados

In [56]:
df['description'] = df['description'].apply(replace_newlines).apply(remove_minutes).apply(remove_links).apply(remove_extra_words)


In [41]:
df['description'][500]

'Conferencia de prensa matutina, desde Campeche. Jueves 25 de marzo 2021 | Presidente AMLO.   Inicio de transmisión  Comienza la conferencia de prensa del presidente Andrés Manuel López Obrador  Informe de seguridad del estado de Campeche  Sesión de preguntas y respuestas  Realizamos la conferencia de prensa matutina y la reunión del Gabinete de Seguridad en Campeche, la entidad federativa con menor incidencia delictiva del país. Esto se debe al trabajo de gobierno, al apoyo de la población, a la coordinación entre autoridades y a la atención a las causas de la violencia e inseguridad.  De acuerdo con el análisis del último año, en Campeche disminuyó la incidencia delictiva total en 14.41 por ciento, el feminicidio en 57.14 por ciento, la extorsión en 48 por ciento, el robo de vehículo en 38.89 por ciento, la violencia familiar en 31.58 por ciento, el robo a transeúnte en 2.44 por ciento y la trata de personas en un 100 por ciento.   En los delitos del fuero federal, en el periodo ener

In [35]:
df

,published_at,title,description
0,2023-04-26T06:07:35Z,"Conferencia de prensa matutina, desde Palacio ...","Conferencia de prensa matutina, desde Palacio ..."
1,2023-04-25T04:45:24Z,"Conferencia de prensa matutina, desde Palacio ...","Conferencia de prensa matutina, desde Palacio ..."
2,2023-04-24T12:44:41Z,Conferencia de prensa matutina desde Palacio N...,"Conferencia de prensa matutina, desde Palacio ..."
3,2023-04-21T05:16:34Z,Seguridad y bienestar mejoran en Veracruz. Con...,"Conferencia de prensa matutina, desde Palacio ..."
4,2023-04-20T11:50:16Z,Recursos de venta de avión serán para construi...,"Conferencia de prensa matutina, desde Palacio ..."
...,...,...,...
1082,2019-03-08T21:59:25Z,Conferencia matutina: Avances del Gabinete de ...,"Conferencia de prensa matutina, desde Palacio ..."
1083,2019-03-08T21:59:25Z,Hoy anunciamos la terna de candidatos a minist...,Cuarta conferencia de prensa matutina desde el...
1084,2019-03-08T21:59:25Z,Tercera conferencia de prensa matutina desde P...,Tercera conferencia de prensa matutina desde e...
1085,2019-03-08T21:59:25Z,Diálogo con periodistas desde Palacio Nacional,Segunda conferencia de prensa matutina del pre...


## Separación por semana

Se recorre el data frame

In [49]:
df['published_at']

0       2023-04-26T06:07:35Z
1       2023-04-25T04:45:24Z
2       2023-04-24T12:44:41Z
3       2023-04-21T05:16:34Z
4       2023-04-20T11:50:16Z
                ...         
1082    2019-03-08T21:59:25Z
1083    2019-03-08T21:59:25Z
1084    2019-03-08T21:59:25Z
1085    2019-03-08T21:59:25Z
1086    2019-03-08T21:59:25Z
Name: published_at, Length: 1087, dtype: object

In [50]:
clean_dates = []

for dates in df['published_at']:
    clean_dates.append( datetime.strptime(dates[:10], '%Y-%m-%d') )
df['clean_dates'] = clean_dates

In [52]:
df = df.drop('published_at', axis=1) # Ahorra borramos las fechas que no nos sirven

In [60]:
df['clean_dates'] = pd.to_datetime(df['clean_dates'])

Ya tenemso los datos como fechas tipo <code>datetime</code>

In [66]:
df = pd.read_csv('mañaneras.csv')

In [69]:
df

,title,description,clean_dates
0,"Conferencia de prensa matutina, desde Palacio ...","Conferencia de prensa matutina, desde Palacio ...",2023-04-26
1,"Conferencia de prensa matutina, desde Palacio ...","Conferencia de prensa matutina, desde Palacio ...",2023-04-25
2,Conferencia de prensa matutina desde Palacio N...,"Conferencia de prensa matutina, desde Palacio ...",2023-04-24
3,Seguridad y bienestar mejoran en Veracruz. Con...,"Conferencia de prensa matutina, desde Palacio ...",2023-04-21
4,Recursos de venta de avión serán para construi...,"Conferencia de prensa matutina, desde Palacio ...",2023-04-20
...,...,...,...
1082,Conferencia matutina: Avances del Gabinete de ...,"Conferencia de prensa matutina, desde Palacio ...",2019-03-08
1083,Hoy anunciamos la terna de candidatos a minist...,Cuarta conferencia de prensa matutina desde el...,2019-03-08
1084,Tercera conferencia de prensa matutina desde P...,Tercera conferencia de prensa matutina desde e...,2019-03-08
1085,Diálogo con periodistas desde Palacio Nacional,Segunda conferencia de prensa matutina del pre...,2019-03-08


In [68]:
df = df.drop('Unnamed: 0', axis=1)

In [39]:
df.to_csv('mañaneras.csv')

In [43]:
df['clean_dates']

0       2023-04-26
1       2023-04-25
2       2023-04-24
3       2023-04-21
4       2023-04-20
           ...    
1082    2019-03-08
1083    2019-03-08
1084    2019-03-08
1085    2019-03-08
1086    2019-03-08
Name: clean_dates, Length: 1087, dtype: object

Convertimos el Data Frame en una serie de tiempo con <code>pd.set_index</code>.
De esta manera las fechas ahora serán los índices de el dataframe

In [71]:
df.set_index('clean_dates', inplace=True)

In [75]:
df

,title,description
clean_dates,,
2023-04-26,"Conferencia de prensa matutina, desde Palacio ...","Conferencia de prensa matutina, desde Palacio ..."
2023-04-25,"Conferencia de prensa matutina, desde Palacio ...","Conferencia de prensa matutina, desde Palacio ..."
2023-04-24,Conferencia de prensa matutina desde Palacio N...,"Conferencia de prensa matutina, desde Palacio ..."
2023-04-21,Seguridad y bienestar mejoran en Veracruz. Con...,"Conferencia de prensa matutina, desde Palacio ..."
2023-04-20,Recursos de venta de avión serán para construi...,"Conferencia de prensa matutina, desde Palacio ..."
...,...,...
2019-03-08,Conferencia matutina: Avances del Gabinete de ...,"Conferencia de prensa matutina, desde Palacio ..."
2019-03-08,Hoy anunciamos la terna de candidatos a minist...,Cuarta conferencia de prensa matutina desde el...
2019-03-08,Tercera conferencia de prensa matutina desde P...,Tercera conferencia de prensa matutina desde e...


In [72]:
weekly_ts = df.resample('W').sum()

In [80]:
weekly_ts['description'].to_csv('mañaneras.csv')